(sec-floating-point)=
# Floating Point Numbers

There is no way to express real numbers in discrete systems. For example, we cannot express any irrational number using a finite number of letters 0-9.  Therefore, we express real number approximately using scientific notation such as $1.32567 \times 10^{12}$.  Similarly digital computers use so-called *floating point* representation.  $1.32567 \times 10^{12}$ is expressed as $1.32567E12$.  Since scientific computation relies on the properties of floating point, we need to unser stand them.{cite}`Goldberg1991`

A *single precision* floating point stores a real number in a 32-bit string, of which 24 bits are used for mantissa.  The corresponding significant figure is $\log_{10} 2^{24} \approx 7$.  The exponent part is $2^{2^7}=2^{-128}$ to $2^{2^7-1}=2^{127}$ which is approximately $10^{-38}$ to $10^{+38}$.  One bit is used for the sign.  This method allows two different expressions of zero, $+0.$ and $-0.$.  Inside the computers, they are treated as two different numbers.
Usually, the single precision is not accurate enough for computational physics and thus we should avoid it.

A *double precision floating point* uses a 64-bit string, 54 bits for mantissa and 10 bits for exponent as shown in {numref}`fig-double-float`.  The largest value the mantissa can express  is $2^{53} = 9007,199,254,740,992$, which corresponds to significant figure 16.  The maximum exponent part is between $2^{-2^9} = 2^{-512} \approx 10^{-308}$ and $2^{2^9-1} = 2^{511} \approx 10^{308}$.  Modern computers implement more advanced floating point arithmatics known as IEEE 754. Common CPUs used in desktop and laptop computers are capable of double precision floating point arithmetic. Some advanced computers are equipped with special arithmatic engine capable ofr 128-bits floating point arithmatics. The default size of floating point in python is 64, which is good enough for most of numerical calculation in physics. 
Modern computers implement more advanced floating point arithmatics known as [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754#2019). Common CPUs used in desktop and laptop computers are capable of double precision floating point arithmetic.  Some advanced computers are equipped with special arithmatic engine capable ofr 128-bits floating point arithmatics. The default size of floating point in python is 64, which is good enough for most of numerical calculation in physics.

```{figure} double-float.png
---
name: fig-double-float
---
64-bit string for floating point expression.  The last bit is used for the sign and 11 bits from $b_{52}$ to $b_{62}$ express the exponent.  The remaining 52 bits express the mantissa.
```

[^denormalized]: There is a slightly better enconding known as *denormalized float*.  The smallest value in the denormalized float method is $4.9406564584124654 \times 10^{-324}$ for double and $1.401298 \times 10^{-45}$ for single  which is smaller than the smallest value in the standard floating point method.  Most of modern computer programming language use denormalized float for very small number.  However, if the smallest possible value is asked, the system still returns the value in the standard floating point.

## The range of floating points

As discussed above, floating point has a finite range based on the size of bit string.  In most computers, the range is

|  Type  | Minimum value   |  Maximum value |
| ------ | -------------   | -------------- |
| single | 1.175494351E-38 | 3.402823466E+38|
| double | 2.2250738585072014E-308 | 1.7976931348623158E+308|

You don't have to memorize these numbers since pythonknows them.  The follwoing example extract those information from python. 


---

**Example 1.4.1**: Range of floating point numbers

Let us try to find the largest and smallest *positive* numbers in your computer system.  We use the `float_info` class in `sys` library.

In [1]:
# load sys package for system information
import sys

fmin = sys.float_info.min
fmax = sys.float_info.max
print("Smallest float =",fmin)
print(" Largest float =",fmax)


Smallest float = 2.2250738585072014e-308
 Largest float = 1.7976931348623157e+308


## Special value "inf"

If the value exceeds the max, python outputs "inf".  Although it is not real infinity, python thinks it is.  Getting inf means your calculation failed due to *overflow error*.

---

**Example 1.4.2**: number above fmax

Find what is the outcome of a number larger than the maximum or smaller than minimum.

In [2]:
print("2 x fmax =", 2*fmax*2)

2 x fmax = inf


## Special value "nan"

If mathematical operation is undefined, python just outputs "nan" whcih stands for "not a number".


---

**Example 1.4.3**: square root of -1.

$\sqrt{-1}$ is not defined within floating point. (The squre root function is defined as floating point and thus it does not understand complex number.)  Python issues a runtim warning message.

In [1]:
# square root is not available in python core.
# we use numpy
import numpy as np

np.sqrt(-1)

/tmp/ipykernel_11450/965727510.py:5: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(-1)


nan

## Overflow errors

When the output of a calculation exceeds the maximum of floating point, you need to find a better algorithm to compute it.  Thre is no universal mitigation of "inf".  The following example is often used in statistical physcs.

---

**Example 1.4.4** 

Factorial of a large integer is astronomically large. For example, 1000!.  Let try to write it down. (We are wasting space.)

In [11]:
# Here we use math package to compute a large factorial.
# (numpy math.factorial has been deprecated.)
import math

# 1000! using math factorial function
math.factorial(1000)


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

which is practically useless.  It is diffuclt even to see how big the value is.  So, we want to express it in scientific notation $a \times 10^{b}$. If you try to convert the above integer number to a floating point number using `float(math.factorial(1000))`, the number is obviously too large and the conversion causes overflow error.  We need to calculate the scientific notation manually. In order to find the mantissa $a$ and exponent $b$, first we evaluate $\log N!$ as follows.

$$
\begin{eqnarray}
y &=& \log(N!) = \log(1 \cdot 2 \cdot 3 \cdots N-1 \cdot N) \\
&=& \log(1)+\log(2)+\log(3)+\cdots + \log(N-1)+\log(N)
\end{eqnarray}
$$

which is just a sum of small number.  Once we found $y$, the factorial can be obtained by $n! = e^y$.  However, it is still not in scientific notation.  First we change the base from $e$ to $10$ as $e^y  = 10^z $,  where $z = y \log_{10}(e)$.  Then, $n! = 10^z$.
Next we split $z$ to the floor k=$\lfloor z \rfloor$ and the residual $\delta=z - \lfloor z \rfloor$.
Now, we have $n! = 10^{k+\delta} = 10^\delta \times 10^k$ and thus the mantissa is $10^\delta$ and power is $k$.  

In [12]:
# evaluate log(1) + log(2) + ... + log(1000)
y=np.log(np.arange(1,1001)).sum()

# change the base from e to 10
z=np.log10(np.exp(1))*y

# find power
b = int(np.floor(z))

# find mantissa
a = 10**(z-b)

print("power=",b)
print("mantissa=",a)

power= 2567
mantissa= 4.023872600769742


which tells that $1000! \approx 4.0239 \times 10^{2567}$.  You can see that the number is far above the maximum of double precision.


---
Last modified on 02/09/2024 by R. Kawai.